# Подготовка

Это предварительные действия, которые лучше выполнить перед семинаром!

1. Скачать файл с моделью AdaGram для русского языка по [ссылке](https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib)
2. Переместить файл из п. 1 в папку с этой тетрадкой
3. Скачать архив с моделью ELMo для русского языка по [ссылке](http://vectors.nlpl.eu/repository/20/196.zip)
4. Распаковать архив в папку с этой тетрадкой
5. Скачать в папку с этой тетрадкой дополнительные файлы [из репозитория курса](5_WSD)
6. Установить необходимые библиотеки (ячейки ниже)

P.S. Можно проделать всё это в Colab'е, но там есть проблемы с запуском MyStem :(

In [ ]:
!pip install tensorflow Cython matplotlib

In [ ]:
!pip install simple-elmo

In [ ]:
!pip install git+https://github.com/lopuhin/python-adagram.git

In [ ]:
!pip install pymystem3 pymorphy2

In [ ]:
# библиотеки для работы с эмбеддингами
import adagram
from simple_elmo import ElmoModel

# обработка данных и ML
import pandas as pd
from lxml import html
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
from tqdm.notebook import tqdm
from sklearn.metrics import adjusted_rand_score
from sklearn.decomposition import PCA
from sklearn.cluster import *
from collections import Counter
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')


morph = MorphAnalyzer()
token = RegexpTokenizer('\w+')
stops = set(stopwords.words('russian'))

def normalize(text):
    words = [morph.parse(word)[0].normal_form for word in tokenize(text) if word]
    return words

def tokenize(text):
    return token.tokenize(text)

## Адаграм

Word2Vec и многие другие векторные модели сопоставляют 1 вектор. Это значит, что у каждого слова в векторном пространстве только 1 значение. У многозначных слов векторы будут просто каким-то усреднением или обобщением всех его значений. 

В работе https://arxiv.org/pdf/1502.07257.pdf предлагается способ улучшить Skip Gram, так чтобы каждому слову сопоставлялось K различных векторов, так что каждый из них представляет какое-то из его значений. При этом сам параметр K задавать не нужно, модель сама находит нужное количество "значений" для каждого слова.

Изначально этот  подход реализован на julia, но есть реализация на питоне - https://github.com/lopuhin/python-adagram

In [ ]:
#!curl "https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib" > all.a010.p10.d300.w5.m100.nonorm.slim.joblib

In [ ]:
vm = adagram.VectorModel.load('all.a010.p10.d300.w5.m100.nonorm.slim.joblib')

Посмотрим на значения каких-нибудь слов: эти "значения"  задаются индексами, у каждого есть вероятность.

In [ ]:
vm.word_sense_probs('вечер')

Посмотрим какие слова близки к каждому из значений.

In [ ]:
vm.sense_neighbors('вечер', 0)

In [ ]:
vm.sense_neighbors('вечер', 1)

In [ ]:
vm.sense_neighbors('вечер', 2)

In [ ]:
context = "Сегодня вечером я иду в гости."

ВОПРОС! Как можно дизамбигуировать контексты, используя соседей для каждого значения?

Можно посмотреть на все слова у которых есть хотя бы 2 устойчивых значения.

In [ ]:
ambiguous = []
for i, word in enumerate(vm.dictionary.id2word):
    probs = vm.word_sense_probs(word)
    if len(probs) > 1:
        ambiguous.append(word)
print(ambiguous[:50])

Дизамбигуация AdaGram основана на вычислении вероятности вектора каждого значения в заданном контексте. 

Функция `model.disambiguate` возвращает массив вероятностей для всех значения данного слова:

In [ ]:
means = vm.disambiguate('вечер', normalize("Ради любви родителей, ради того, чтобы они снова также танцевали в их гостиной, наслаждаясь милыми семейными"))

In [ ]:
means

Чтобы выяснить, какое значение выбрал AdaGram, нужно найти индекс вектора с максимальной вероятностью:

In [ ]:
vm.sense_neighbors('вечер', np.argmax(means))

In [ ]:
means = vm.disambiguate('вечер', normalize("абонемент № 19 \"Камерные\" включает в себя и концерт лауреата последнего Конкурса Чайковского"))

In [ ]:
vm.sense_neighbors('вечер', np.argmax(means))

## WSD / WSI
Разрешение семантической/лексической неоднозначности/омонимии

Проверим, насколько хорошо выбирается значение на данных с [соревнования Диалога](http://www.dialog-21.ru/evaluation/2018/disambiguation/) (переиспользую [baseline](https://github.com/nlpub/russe-wsi-kit) соревнования)
(А [вот](http://www.dialog-21.ru/media/5077/bolshinaasplusloukachevitchnv-108.pdf), кстати, новая статья о генерации обучающих данных для WSD)

**NB!** Большая модель AdaGram для русского языка, которую мы используем, обучена на корпусе с нормализацией *mystem*. Так что немного модифицируем нашу функцию нормализации.

In [ ]:
mystem = Mystem()

def lemmatized_context(s):
    return [w.lower() for w in mystem.lemmatize(" ".join(tokenize(s)))]

def disambiguate(model, word, context):
    word, _ = lemmatized_context(word)
    probs = model.disambiguate(word, lemmatized_context(context))
    return 1 + probs.argmax()


Небольшой подкорпус RUSSE - всего 4 неоднозначных слова:

In [ ]:
df = pd.read_csv('5_WSD/train.baseline-adagram.csv', sep='\t')

In [ ]:
pd.set_option('display.max_colwidth', 1000)

In [ ]:
df.sample(frac=0.05)

In [ ]:
df['predict_sense_id'] = [disambiguate(vm, word, context)
                          for word, context in tqdm(zip(df['word'], df['context']), total=len(df))]

In [ ]:
df.groupby('word').sum()

In [ ]:
per_word = df.groupby('word').apply(
    lambda f: adjusted_rand_score(f['gold_sense_id'], f['predict_sense_id'])
).to_frame('ARI')
per_word_ari = per_word['ARI']
print('Mean word ARI: %.4f' % np.mean(per_word_ari))

In [ ]:
per_word

В качестве метрики используется [Adjuster Rand Index](https://en.wikipedia.org/wiki/Rand_index), а [вот ссылка на документацию](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.adjusted_rand_score.html).

## Contextualized embeddings
[ELMo](https://arxiv.org/pdf/1802.05365.pdf) — модель, которая позволяет получить контекстуальный (contextualized) вектор слова  - учитывающий контекст:

<img src="https://jalammar.github.io/images/Bert-language-modeling.png" alt="elmo" width="400"/>

(Подробнее в слайдах лекции)

С ELMo легко работать с помощью библиотеки simple_elmo.
Скачиваем модель [отсюда](http://vectors.nlpl.eu/repository/20/196.zip) (см. инструкции выше).

In [ ]:
model = ElmoModel()
model.load("196")

У модели есть метод `get_elmo_vectors`, который принимает на вход массив контекстов - и возвращает массив матриц векторов - для каждого слова каждого входного текста.
Нормализуем предложение и достанем контекстуализированный вектор неоднозначного слова.

In [ ]:
sentence = "многочисленные укрепленные монастыри также не являлись замками как таковыми — это были крепости"
tokens = normalize(sentence)
word_idx = tokens.index("замок")
word_vector = model.get_elmo_vectors([tokens])[0][word_idx]  # 0 - индекс контекста

In [ ]:
word_vector

Чтобы каждый раз не повторять эту процедуру, обернём в свою функцию.

In [ ]:
def get_elmo_vectors(word, contexts, model):
    tokens = [normalize(c) for c in contexts]
    all_vectors = model.get_elmo_vectors(tokens)
    word_vecs = []
    for i in range(len(contexts)):
        try:
            word_vecs.append(all_vectors[i][tokens[i].index(word)])
        except ValueError:  # если нормализация накосячила и лемму не найти
            continue
    return word_vecs

Попробуем сначала нарисовать, какие получаются вектора одного и того же слова в разных контекстах (пропустим немного заранее заготовленной магии matplotlib и PCA)

In [ ]:
def dim_reduction(X, n):
    pca = PCA(n_components=n)
    print("size of X: {}".format(X.shape))
    results = pca.fit_transform(X)
    print("size of reduced X: {}".format(results.shape))

    for i, ratio in enumerate(pca.explained_variance_ratio_):
        print("Variance retained ratio of PCA-{}: {}".format(i+1, ratio))

    return results

In [ ]:
def plot(word, contexts, labels, reduced_X, context_size=5):
    plt.clf()
    fig, ax = plt.subplots()
    fig.set_size_inches(12, 10)
    colors = ['ro', 'bo', 'yo', 'go', 'co']
    label_color = {}
    for i, l in enumerate(set(labels)):
        label_color[l] = colors[i]

    i = 0
    points = []
    tokens_list = []
    for j, (c, l) in enumerate(zip(contexts, labels)):
        tokens = normalize(c)
        tokens_list.append(tokens)
        color = label_color[l[0]]
        for k, w in enumerate(tokens):
            if w == word:  # рисуем первое вхождение слова в контексте
                ax.plot(reduced_X[j, 0], reduced_X[j, 1], color)
                points.append((j, k, reduced_X[j, 0], reduced_X[j, 1]))
                break
            i += 1

    for p in points:
        s = tokens_list[p[0]]
        text = ' '.join(s[max(0, p[1] - context_size):min(p[1] + context_size, len(s))])

        # bold the word of interest in the sentence
        text = text.replace(word, r"$\bf{" + word + "}$")

        plt.annotate(text, xy=p[2:])
    ax.set_xlabel("PCA 1")
    ax.set_ylabel("PCA 2")

И возьмем датасет побольше:

In [ ]:
train = pd.read_csv('5_WSD/train.csv', sep='\t')

Все слова датасета:

In [ ]:
print(set(train['word'].to_list()))

In [ ]:
train[train['word'] == 'замок']

In [ ]:
sentences_1 = train[train['word']=='замок'][train['gold_sense_id']=='1'].sample(5, random_state=21)
sentences_2 = train[train['word']=='замок'][train['gold_sense_id']=='2'].sample(5, random_state=21)

In [ ]:
sentences = list(sentences_1['context']) + list(sentences_2['context'])
labels = list(sentences_1['gold_sense_id']) + list(sentences_2['gold_sense_id'])

In [ ]:
sentences

In [ ]:
X = get_elmo_vectors('замок', sentences, model)

In [ ]:
X = np.array(X)
X.shape

In [ ]:
X_reduce = dim_reduction(X=X, n=2)

In [ ]:
sentences

In [ ]:
plot('замок', sentences, labels, X_reduce)

Что можно сделать с этими векторами в целях WSD?
* классификация
* кластеризация


# Задание (в группах)

Вариант 1:

- Напишите функцию, которая вычисляет центроид (средний вектор) каждого значения данного слова по всем контекстам из `train`.
- Напишите вторую функцию, которая принимает на вход слово и произвольный контекст с этим словом, а возвращает индекс значения слова в этом контексте: вычисляем контекстный вектор, сравниваем с центроидами значений, выбираем ближайшее. Можно также вывести насколько контекстов для этого значения из обучающего множества.

Вариант 2:
- Выберите один их методов кластеризации:
  - [K-Means]()
  - [Affinity Propagation]()
  - или любой другой приятный вам метод из [sklearn.cluster]()
- Напишите функцию, которая будет принимать на вход слово, кластеризовать его контексты из `train` и вычислять ARI по сравнению с эталонной разметкой значений.
- (*) Если останется время, можно нарисовать получившуюся кластеризацию - с помощью функции `plot`, которая определена выше.